# Aplicações do Finbert-PT-BR

In [ ]:
from transformers import (AutoTokenizer, BertForSequenceClassification, AutoModelForSequenceClassification, pipeline)
from sklearn.metrics import classification_report
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# finbert_pt_br_tokenizer = AutoTokenizer.from_pretrained("lucas-leme/FinBERT-PT-BR")
# finbert_pt_br_model = AutoModelForSequenceClassification.from_pretrained("lucas-leme/FinBERT-PT-BR")

# pipe = pipeline(task='text-classification', model=finbert_pt_br_model, tokenizer=finbert_pt_br_tokenizer)
# pipe(['Hoje a bolsa caiu', 'Hoje a bolsa subiu'])

pipe = pipeline("text-classification", model="lucas-leme/FinBERT-PT-BR")
print(pipe(['Hoje a bolsa caiu', 'Hoje a bolsa subiu']))

In [ ]:
df = pd.read_csv('data/finbert_ptbr_test.csv')